# Assignment 1: The Jordan Network

*Author:* Thomas Adler

*Copyright statement:* This  material,  no  matter  whether  in  printed  or  electronic  form,  may  be  used  for  personal  and non-commercial educational use only.  Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.

## Exercise 1: Initializing the network
Consider the Jordan network (with $f(x) = \tanh(x) = (e^x - e^{-x})(e^x + e^{-x})^{-1}$ and $\varphi(x) = \sigma(x) = (1+e^{-x})^{-1}$ and transposed weight matrices compared to the lecture notes)
$$
s(t) = W x(t) + R \hat y(t-1) \\
a(t) = \tanh(s(t)) \\
z(t) = V a(t) \\
\hat y(t) = \sigma(z(t))
$$
for $t \in \mathbb{N}, x(t) \in \mathbb{R}^{D}, s(t) \in \mathbb{R}^{I}, a(t) \in \mathbb{R}^{I}, z(t) \in \mathbb{R}^K, \hat y(t) \in \mathbb{R}^K$ and $W, R, V$ are matrices of appropriate sizes and $\hat y(0) = 0$. 

Write a function `init` that takes a `model` and integers $D, I, K$ as arguments and stores the matrices $W, R, V$ as members `model.W`, `model.R`, `model.V`, respectively. The matrices should be `numpy` arrays of appropriate sizes and filled with random values that are uniformly distributed between -0.01 and 0.01. 

In [ ]:
%matplotlib inline
import numpy as np
from numpy import random 
from scipy.special import expit as sigmoid

class Obj(object):
    pass

model = Obj()
T, D, I, K = 10, 3, 5, 1

#W [IxD] dimension of current layer is s & prev layer (x) but .T to be @x, W@x
#R [IxK] current layer is s & prev is y hat
#V [KxI] current layer is z & prev is a

def init(model, D, I, K,T):
    ########## YOUR SOLUTION HERE ##########
    model.W = np.random.uniform(low=-0.01, high=0.01, size=(I,D))
    model.R = np.random.uniform(low=-0.01, high=0.01, size=(I,K))
    model.V = np.random.uniform(low=-0.01, high=0.01, size=(K,I))
    

Obj.init = init 
model.init(D, I, K) 

## Exercise 2: Numerical stability of the binary cross-entropy loss function

We will use the binary cross-entropy loss function to train our RNN, which is defined as 
$$
L(\hat y, y) = -y \log \hat y - (1-y) \log (1-\hat y) \quad \text{where} \quad \hat y = \sigma(z) = \frac{1}{1+e^{-z}}
$$
is the sigmoid function. Its argument $z$ is called *logit*. For reasons of numerical stability it is better to let the model emit the logit and incorporate the sigmoid function into the loss function. Explain why this is the case and how we can gain numerical stability by combining the two functions into one. *Hint:* Find a numerically stable version of the function $f(z) = \log(1+e^{z})$ and rewrite $L(z,y)$ in terms of $f(z)$. 

########## YOUR SOLUTION HERE ##########

z is an unbounded linear function that when passed by the sigmoid function is bounded by [0,1].

$$e^{-z} \text {must be emitted from the model since if:}$$
$$z << 0 \rightarrow y \approx 0 \rightarrow log 0 = \infty$$

So, it is more stable to combine operations into 1 layer to take advantage of the log-sum-exp:

$$log \sum_{t=1}^{T} e^z = a + log \sum_{t=1}^{T} e^{z+a}$$

## Exercise 3: The forward pass
Write a function `forward` that takes a `model`, a sequence of input vectors $(x(t))_{t=1}^T$, and a label `y` as arguments. The inputs will be represented as a `numpy` array of shape `(T, D)`. It should execute the behavior of the Jordan network and evaluate the numerically stablilized binary cross-entropy loss at the end of the sequence and return the resulting loss value. Store the sequence of hidden activations $(a(t))_{t=1}^T$ and the sequence of logits $(z(t))_{t=1}^T$ into `model.a` and `model.z`, respectively, using the same representation scheme as for the inputs. 

In [ ]:
import numpy.matlib

def forward(self, x, y):
    ########## YOUR SOLUTION HERE ##########
    self.s = np.zeros((T,I))
    self.a = np.zeros((T,I))
    self.z = np.zeros((T,K))
    self.x = x
    self.y = y
    self.y_hat = np.zeros((T+1,K))
    prev_y_hat=0
    #loss = 0
    
    for t in range(0,T):
        s = (self.W @ x[t]) + (self.R @ self.y_hat[t-1])
        self.s[t] = s
        a = np.tanh(s)
        self.a[t] = a
        z = self.V @ a
        self.z[t] = z
        y_hat = sigmoid(z)
        self.y_hat[t] = y_hat
        prev_y_hat=y_hat
    loss = - y*np.log(y_hat) - (1-y)*np.log(1-y_hat)
        
    loss /= T
    return loss     

Obj.forward = forward
model.forward(np.random.uniform(-1, 1, (T, D)), 1)

## Exercise 4: The computational graph

Visualize the functional graph of the Jordan network unfolded in time. The graph should show at least 3 consecutive time steps. Use the package `networkx` in combination with `matplotlib` to draw a directed graph with labelled nodes and edges. If you need help take a look at [this guide](https://networkx.guide/visualization/basics/). Make sure to arrange the nodes in a meaningful way. 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

########## YOUR SOLUTION HERE ##########
G = nx.DiGraph()

G.add_node("y_hat(0)",pos=(0,0))
G.add_node("y_hat(1)",pos=(3,-2))
G.add_node("y_hat(2)",pos=(5,-2))
G.add_node("y_hat(3)",pos=(7,-2))

G.add_node("a(1)",pos=(3,0))
G.add_node("a(2)",pos=(5,0))
G.add_node("a(3)",pos=(7,0))

G.add_node("x(1)",pos=(3,2))
G.add_node("x(2)",pos=(5,2))
G.add_node("x(3)",pos=(7,2))

G.add_edge("y_hat(0)", "a(1)", weight="R")
G.add_edge("a(1)", "y_hat(1)", weight="V")
G.add_edge("y_hat(1)", "a(2)", weight="R")
G.add_edge("a(2)", "y_hat(2)", weight="V")
G.add_edge("y_hat(2)", "a(3)", weight="R")
G.add_edge("a(3)", "y_hat(3)", weight="V")
G.add_edge("x(1)", "a(1)", weight="W")
G.add_edge("x(2)", "a(2)", weight="W")
G.add_edge("x(3)", "a(3)", weight="W")

labels = nx.get_edge_attributes(G, 'weight')
positions = nx.get_node_attributes(G, 'pos')

pos = nx.planar_layout(G)

nx.draw_networkx(G, with_labels = True, pos=positions, node_size=900)
nx.draw_networkx_edges(G, pos= positions, edgelist=G.edges())
nx.draw_networkx_edge_labels(G, pos= positions, edge_labels=labels)
plt.show()

## Exercise 5: Derivative of the loss

Calculate the derivative of the binary cross-entropy loss function $L(z, y)$ with respect to the logit $z$.

########## YOUR SOLUTION HERE ##########
$$
L(\hat y, y) = -y \log \hat y - (1-y) \log (1-\hat y) \quad $$

$$\boldsymbol{\frac{\partial L} { \partial \hat{y}} = \frac{-y}{\hat{y}}  \hat{y}' + \frac{-(1-y)}{1 - \hat{y}} (1 - \hat{y})'}$$

$$= \frac{-y}{\hat{y}} \hat{y}' + \frac{1 - y}{1 - \hat{y}} (1 - \hat{y})'$$

$$= \big( \frac{y}{\hat{y}} + \frac{1 - y}{1 - \hat{y}} \big) \hat{y}'$$

$$= \big( \frac{-y + y\hat{y} + \hat{y} - y\hat{y}}{\hat{y}(1 - \hat{y})} \big) \hat{y}'$$

$$= \big( \frac{\hat{y} - y}{\hat{y}(1 - \hat{y})} \big) \hat{y}' $$

$$ \frac{\partial L}{\partial z} = \sigma(z) (1 - \sigma(z)) \frac{\sigma(z) - y}{\sigma(z)(1 - \sigma(z))}$$

$$= \sigma(z) - y$$

## Exercise 6: Gradients with respect to network parameters
Compute gradients for the weights of the Jordan network analytically. That is, derive backpropagation through time for the Jordan network. To do this, it is crucial to first find the derivative w.r.t. the network outputs $\psi^\top(t) = \partial L / \partial z(t)$ and hidden pre-activations $\delta^\top(t) = \partial L / \partial s(t)$. We use the shorthand notations $L = \sum_{t=1}^T L(t)$ and $L(t) = L(y(t), \hat y(t))$ for convenience. We use numerator-layout notation for partial derivatives (like in the lecture notes) which lets us multiply the chain rule from left to right. 

Compute the gradients $\psi^\top(t), \delta^\top(t), \nabla_W L, \nabla_R L, \nabla_V L$. *Hint:* Take a look at the  graph from the previous exercise to see the functional dependencies. 

########## YOUR SOLUTION HERE ##########

1. $\psi^\top(t) = \frac{\partial {L}}{\partial{z(t)}} = \frac{\partial {L}}{\partial{\hat y(t)}} . \frac{\partial {\hat y(t)}}{\partial{z(t)}} = \hat y - y $


        
2. $\delta^\top(t)^T = \frac{\partial L}{\partial s(t)} = \frac{\partial {L}}{\partial{\hat y(t)}} . \frac{\partial {\hat y(t)}}{\partial{z(t)}} . \frac{\partial z(t)}{\partial a(t)} . \frac{\partial a(t)}{\partial s(t)} = \Big[ \frac{\partial {L}}{\partial{\hat y(t)}} + \frac{\partial{L}}{\partial{s(t+1)}}\frac{\partial{s(t+1)}}{\partial{\hat y(t)}} \Big] . \frac{\partial {\hat y(t)}}{\partial{z(t)}} . \frac{\partial z(t)}{\partial a(t)} . \frac{\partial a(t)}{\partial s(t)}= (\psi^\top(t) + \delta(t+1) .R) . V (1-tanh^2s(t)) $
$= \psi^\top(t) . V (1-a(t)^2)$
    
    
    
3. $\nabla_W L$ = ${\frac{\partial L}{\partial r}}^T = \sum {\frac{\partial L}{\partial s(t)}} . {\frac{\partial s(t)}{\partial W}} = \sum {\delta(t) x(t)}^T $


    
4. $\nabla_R L$ = ${\frac{\partial L}{\partial w}}^T =  \sum {\frac{\partial L}{\partial s(t)}} . {\frac{\partial s(t)}
{\partial R}} = \sum {\delta(t) \hat{y}(t-1)}^T $



5. $\nabla_V L$ = ${\frac{\partial L}{\partial v}}^T = \sum {\frac{\partial L}{\partial z(t)}} . {\frac{\partial z(t)}{\partial V}} = \sum {\psi(t) a(t)}^T $

## Exercise 7: The backward pass
Write a function `backward` that takes a model `self` as argument. The function should compute the gradients of the loss with respect to all model parameters and store them to `model.dW`, `model.dR`, `model.dV`, respectively. 

In [ ]:
def backward(self):
    ########## YOUR SOLUTION HERE ##########
    y = self.y
    x = self.x
    R = self.R
    V = self.V
    
    self.dW = 0
    self.dR = 0
    self.dV = 0
    self.delta = np.zeros((T,I))
    
    for t in range(T-1, -1, -1):
        psi = (self.y_hat[t] - self.y)
        
        self.delta[t] = (psi+ self.delta[t-1] @ self.R) @ self.V @ np.diag(1-(self.a[t])**2)
            
        self.dW += np.outer(self.delta[t], self.x[t].T)
        self.dR += np.outer(self.delta[t], self.y_hat[t-1].T)
        self.dV += np.outer(psi, self.a[t].T)

Obj.backward = backward
model.backward()

## Exercise 8: Gradient checking
Write a function `grad_check` that takes a model `self`, a float `eps` and another float `thresh` as arguments and computes the numerical gradients of the model parameters according to the approximation
$$
f'(x) \approx \frac{f(x + \varepsilon) - f(x - \varepsilon)}{2 \varepsilon}.
$$
If any of the analytical gradients are farther than `thresh` away from the numerical gradients the function should throw an error. 

In [ ]:
def grad_check(self, eps, thresh):
    ########## YOUR SOLUTION HERE ##########
    W = self.W
    R = self.R
    V = self.V
    
    weights = [[W,'W'],[R,'R'],[V,'V']]
    
    def check_sign(m, name):
        m_grad_plus = np.zeros(m.shape)
        m_grad_minus = np.zeros(m.shape)
        
        for sign in (1,-1):
            for i in range (0, m.shape[0]):
                for j in range (0, m.shape[1]):
                    m_eps = np.copy(m)
                    m_eps[i,j] += sign*eps
                
                    if(name=='W'):
                        model.W = np.copy(m_eps)
                    elif (name=='R'):
                        model.R = np.copy(m_eps)
                    elif (name=='V'):
                        model.V = np.copy(m_eps)
                    
                    if(sign==1):
                        m_grad_plus[i,j] = self.forward(self.x,self.y)
                    elif(sign==-1):
                        m_grad_minus[i,j] = self.forward(self.x,self.y)
        
        num_grad = (m_grad_plus - m_grad_minus)/(2*eps)
        
        if(name=='W'):
            real_grad = self.dW
        elif (name=='R'):
            real_grad = self.dR
        elif (name=='V'):
            real_grad = self.dV
        
        diff = np.max(num_grad - real_grad)
       # num= np.linalg.norm(num_grad - real_grad)
       # den= np.linalg.norm(real_grad) + np.linalg.norm(num_grad)
       # diff=num/den
        return diff
    
    for w,name in weights:
        diff = check_sign(w,name)
        
        if (diff<thresh):
            print("Grad check for: " +name+ " passed:", diff)
        else:
            print("Grad check for: " +name+ " failed:", diff)
                    

Obj.grad_check = grad_check
model.grad_check(1e-7, 1e-7)

## Exercise 9: Parameter update

Write a function `update` that takes a `model` and a float argument `eta`, which represents the learning rate. The method should implement the gradient descent update rule $\theta \gets \theta - \eta \nabla_{\theta}L$ for all model parameters $\theta$.

In [ ]:
def update(self, eta):
    ########## YOUR SOLUTION HERE ##########
    for t in range(0,T):
        self.R = self.R - eta*self.dR
        self.V = self.V - eta*self.dV 
        self.W = self.W - eta*self.dW

Obj.update = update
model.update(0.001)

## Exercise 10: Data generation

There are two classes, both occurring with probability 0.5. There is one input unit. Only the first sequence element conveys relevant information about the class. Sequence elements at positions $t > 1$ are generated by a Gaussian with mean zero and variance 0.2. The first sequence element is 1.0 (-1.0) for class 1 (2). Target at sequence end is 1.0 (0.0) for class 1 (2)

Write a function `generate_data` that takes an integer `T` as argument which represents the sequence length. Seed the `numpy` random generator with the number `0xDEADBEEF`. Implement the Python generator pattern and produce data in the way described above. The input sequences should have shape `(T, 1)` and the target values should have shape `(1,)`.

In [ ]:
random.seed(0xDEADBEEF)

def generate_data(T):
    ########## YOUR SOLUTION HERE ##########
    data = np.zeros((T,1),dtype=float)
    
    for i in range(1,T):
        data[i] = np.random.normal(0, np.sqrt(0.2))
    
    values=[-1.0,1.0]
    P=[0.5,0.5]
    data[0]=np.random.choice(values,replace=False, p=P)

    if data[0]==1.0:
        y=1.0
    elif data[0]==-1.0:
        y=0.0
    
    target = np.array([y])
        
    #print(data.T,target)
    return (data,target)

data = generate_data(5)

## Exercise 11: Network training

Train a Jordan network with 32 hidden units. Start with input sequences of length one and tune the leraning rate and the number of update steps. Then increase the sequence length by one and tune the hyperparameters again. What is the maximal sequence length for which the Jordan network can achieve a performance that is better than random? Visualize your results. 

In [ ]:
########## YOUR SOLUTION HERE ##########

In [ ]:
import matplotlib.pyplot as plt 
def train(self, epochs=1, batchsize=1, eta=0.001,sequenceLen=1):
    losses=[]
    for i in range(epochs):
        loss=0
        for j in range(batchsize):
            xx,y=generate_data(sequenceLen)
            loss+=self.forward(xx,y)
            self.backward()
            self.update(eta=eta)
            losses.append(loss)  
            
    return losses

        
Obj.train = train
train32 = Obj()
T,D,I,K = 1,1,32,1
train32.init(D,I,K,T)

In [ ]:

Ts = [1,32,64]
D,I,K = 1,32,1
epochs = [550,2000,4100]
etas = [0.01,0.1,0.6]

plt.rcParams["figure.figsize"] = [20,20]
fig = plt.figure()
fig,axs = plt.subplots(3,3)

axs[1,0].set_title('Axis [1,0]')
axs[1,1].set_title('Axis [1,1]')

for T, i in zip(Ts, range(0,len(etas)*len(Ts),len(etas))):
    for eta, j in zip(etas, range(len(etas))):
        for epoch,k in zip(epochs, range(len(epochs))): 
            train32.init(D,I,K,T)
            losses = train32.train(epochs=epoch, batchsize=1, eta=eta, sequenceLen=T)
            
            axs[i+j,0].set_title(f'T={T}, eta={eta} and {epoch}')
            axs[i+j,0].set_xlabel('Iteration')
            axs[i+j,0].set_ylabel('Loss')
            
            axs[i+j,1].set_title(f'T={T}, eta={eta} and {epoch}')
            axs[i+j,1].set_xlabel('Iteration')
            axs[i+j,1].set_ylabel('Loss')
            
            axs[i+j,2].set_title(f'T={T}, eta={eta} and {epoch}')
            axs[i+j,2].set_xlabel('Iteration')
            axs[i+j,2].set_ylabel('Loss')
            
            
            
            if epoch<epochs[-1]:
                axs[i+j,1].plot(range(1,epoch+1),losses)
                if epoch<epochs[1]:
                    axs[i+j,0].plot(range(1,epoch+1),losses)
                else:
                    axs[i+j,2].plot(range(1,epoch+1),losses)
            
plt.show()